# Chapter 5: 구조화된 프롬프트 vs 비구조화 프롬프트 비교

LLM에 구조화된 프롬프트를 제공한 경우와 제공하지 않은 경우의 응답 차이를 비교하여 프롬프트 엔지니어링의 중요성을 실습합니다.

## 📚 학습 목표
- 구조화된 프롬프트(역할/목표/제약/형식/컨텍스트)와 비구조화 프롬프트의 차이 이해
- RAG 기반 프롬프트 구성 요소 실습
- 프롬프트 구조가 LLM 응답 품질에 미치는 영향 비교

## 📋 실습 구성
1) 헬퍼 함수: `ollama_generate()` 래퍼 및 프롬프트 생성 함수
2) 컨텍스트 정의: 운영체제 관련 문서 청크 시뮬레이션
3) 비구조화 프롬프트: 단순 질문만 전달
4) 구조화 프롬프트: 역할/목표/제약/형식/컨텍스트 포함 프롬프트 생성
5) 비교 실행: 두 프롬프트의 응답을 나란히 비교

> 참고: 본 예제는 RAG 프롬프트 구조의 기본 요소를 다룹니다.


---
## 1️⃣ 실행 전 준비 (Colab)

Colab 환경에서 Ollama를 설치하고 서버를 백그라운드로 구동한 뒤, 사용할 모델을 내려받고 서버 상태를 확인합니다.

**주요 내용:**
- Ollama 설치 스크립트 실행
- `ollama serve` 백그라운드 구동(대기 포함)
- `llama3.1:8b` 모델 pull
- `/api/version`으로 서버 상태 확인

**실행 결과:**
- 설치 완료 메시지와 함께 서버/모델 정보가 출력됩니다.
- 서버 정상 동작 여부 확인 결과 출력
- 로컬 서버 엔드포인트: `http://localhost:11434`


In [ ]:
# ========================================
# 1️⃣ 실행 전 준비 (Colab)
#    - Ollama 설치, 서버 백그라운드 실행, 모델 pull
#    - 헬스 체크: /api/version
# ========================================
import subprocess
import time
import requests

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# Ollama 서버 백그라운드 실행
print("Starting Ollama server ...")
ollama_process = subprocess.Popen(["ollama", "serve"],
                                  stdout=subprocess.DEVNULL,
                                  stderr=subprocess.DEVNULL)
# 서버 시작 대기
time.sleep(5)

# 모델 다운로드
!ollama pull llama3.1:8b

print("\n✅ Setup done.")
print("   Ollama server: http://localhost:11434")
print("   Model: llama3.1:8b (non-streaming)")

# 헬스 체크
print("\n🔄 Checking server status...")
try:
    r = requests.get("http://localhost:11434/api/version", timeout=5)
    r.raise_for_status()
    print("✅ Server ready:", r.json())
except Exception as e:
    print(f"❌ Server error: {e}")


---
## 2️⃣ 헬퍼 함수 및 설정

이 셀에서는 Ollama API 호출용 헬퍼 함수와 프롬프트 생성 함수를 정의합니다.

**주요 내용:**
- `ollama_generate(prompt)`: Ollama API 비스트리밍 호출 래퍼
- `build_structured_prompt()`: 구조화된 프롬프트 생성 함수
- 하이퍼파라미터 설정

**실행 결과:**
- 헬퍼 함수들이 정의되어 사용 가능한 상태가 됩니다.


In [ ]:
# ========================================
# 2️⃣ 헬퍼 함수 및 설정
#    - ollama_generate(): Ollama API 래퍼
#    - build_structured_prompt(): 구조화 프롬프트 생성
# ========================================
import requests
import json

# 하이퍼파라미터 설정
OLLAMA_HOST = "http://localhost:11434"
MODEL_NAME = "llama3.1:8b"
TEMPERATURE = 0.7
TOP_P = 0.9
NUM_PREDICT = 512
REPEAT_PENALTY = 1.1

def ollama_generate(prompt: str) -> str:
    """Ollama API를 통해 비스트리밍 생성 수행"""
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": TEMPERATURE,
            "top_p": TOP_P,
            "num_predict": NUM_PREDICT,
            "repeat_penalty": REPEAT_PENALTY,
        }
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=600)
        r.raise_for_status()
        data = r.json() if isinstance(r.json(), dict) else json.loads(r.text)
        return data.get("response", "")
    except Exception as e:
        return f"[Error] {e}"

def build_structured_prompt(
    role: str,
    goal: str,
    constraints: str,
    format_spec: str,
    context: list[str],
    question: str
) -> str:
    """구조화된 프롬프트 생성
    
    Args:
        role: 모델의 역할 (예: 운영체제 튜터)
        goal: 목표 (예: 근거 기반 답변)
        constraints: 제약 조건 (예: 인용 필수)
        format_spec: 출력 형식 지정
        context: 컨텍스트 청크 리스트
        question: 사용자 질문
    
    Returns:
        완전한 구조화 프롬프트 문자열
    """
    context_text = "\n\n".join([f"[Context {i+1}]\n{chunk}" for i, chunk in enumerate(context)])
    
    prompt = f"""Role:
{role}

Goal:
{goal}

Constraints:
{constraints}

Format:
{format_spec}

Context:
{context_text}

Question:
{question}

Please answer the question considering the role, goal, constraints, format, and context provided above."""
    
    return prompt

print("✅ Helper functions ready.")


---
## 3️⃣ 질문 및 컨텍스트 정의

이 셀에서는 비교 실습에 사용할 운영체제 관련 질문과 가상의 컨텍스트 청크를 정의합니다.

**주요 내용:**
- 사용자 질문: 운영체제 개념에 관한 질문
- 컨텍스트 청크: 질문과 관련된 문서 청크 2-3개 (RAG 검색 결과 시뮬레이션)

**실행 결과:**
- 질문과 컨텍스트가 변수에 저장됩니다.


In [ ]:
# ========================================
# 3️⃣ 질문 및 컨텍스트 정의
#    - 운영체제 관련 질문 생성
#    - 관련 컨텍스트 청크 시뮬레이션
# ========================================

# 사용자 질문
USER_QUESTION = "What is the difference between a process and a thread?"

# 컨텍스트 청크 (RAG 검색 결과 시뮬레이션)
CONTEXT_CHUNKS = [
    """A process is an instance of a program running in an operating system.
Each process has its own independent memory space and does not share memory with other processes.
Inter-process communication (IPC) requires special mechanisms (pipes, sockets, shared memory, etc.).

Key characteristics of a process:
- Owns an independent address space
- High cost for process creation/termination
- Stability: One process crashing does not affect other processes""",

    """A thread is a lightweight execution unit within a process.
Multiple threads within a single process share the same memory space and use code, data, and heap regions together.

Key characteristics of a thread:
- Memory sharing within a process
- Lower creation/context switching cost compared to processes
- Synchronization required: Access control needed for shared memory (mutexes, semaphores, etc.)
- Multithreading is particularly beneficial for I/O-bound tasks""",

    """Key differences between processes and threads:

1. Memory sharing:
   - Process: Independent memory space (each process has its own address space)
   - Thread: Threads within the same process share memory

2. Creation cost:
   - Process: High cost (requires creating a new address space)
   - Thread: Low cost (reuses existing process resources)

3. Communication method:
   - Process: Requires IPC (pipes, sockets, shared memory, etc.)
   - Thread: Direct memory access possible (synchronization required)

4. Fault isolation:
   - Process: An error in one process does not affect other processes
   - Thread: An error in one thread can terminate the entire process"""
]

print(f"질문: {USER_QUESTION}")
print(f"\n컨텍스트 청크 수: {len(CONTEXT_CHUNKS)}")
print("\n컨텍스트 미리보기:")
for i, chunk in enumerate(CONTEXT_CHUNKS, 1):
    print(f"\n[청크 {i}] {chunk[:100]}...")


---
## 4️⃣ 비구조화 프롬프트 생성

이 셀에서는 단순한 질문만 포함한 비구조화 프롬프트를 생성합니다.

**주요 내용:**
- 컨텍스트나 구조 없이 질문만 LLM에 전달
- 이는 일반적인 사용자가 LLM에 질문하는 기본 방식입니다

**실행 결과:**
- 비구조화 프롬프트가 생성되어 출력됩니다.


In [ ]:
# ========================================
# 4️⃣ 비구조화 프롬프트 생성
#    - 단순 질문만 포함
# ========================================

unstructured_prompt = USER_QUESTION

print("=" * 80)
print("비구조화 프롬프트:")
print("=" * 80)
print(unstructured_prompt)
print("=" * 80)


---
## 5️⃣ 구조화 프롬프트 생성

이 셀에서는 역할, 목표, 제약, 형식, 컨텍스트를 모두 포함한 구조화 프롬프트를 생성합니다.

**주요 내용:**
- **역할**: 운영체제 튜터 역할 지정
- **목표**: 근거 기반으로 답변하기
- **제약**: 근거 없으면 '없음' 표시, 인용 1-3개 필수
- **형식**: JSON 스키마 또는 구조화된 형식 지정
- **컨텍스트**: 정의된 컨텍스트 청크 포함

**실행 결과:**
- 완전한 구조화 프롬프트가 생성되어 출력됩니다.


In [ ]:
# ========================================
# 5️⃣ 구조화 프롬프트 생성
#    - 역할/목표/제약/형식/컨텍스트 포함
# ========================================

ROLE = """You are an expert tutor specializing in operating system concepts.
You explain concepts clearly and concisely so students can understand easily,
and you use concrete examples to illustrate concepts."""

GOAL = """Provide evidence-based answers to questions based on the provided context.
Find and cite relevant information from the context in your answers,
and explicitly state when information is not available in the context."""

CONSTRAINTS = """1. Your answer must be based on the provided context.
2. If you can find relevant information in the context, you must include 1-3 citations.
3. If relevant information is not available in the context, you must explicitly state 'The information is not available in the provided context.'
4. Any speculative or uncertain content must be clearly marked."""

FORMAT_SPEC = """Please respond in the following JSON format:

{
  "answer": "Answer to the question (1-2 paragraphs)",
  "citations": [
    {"chunk_id": 1, "quote": "quoted text"},
    {"chunk_id": 2, "quote": "quoted text"}
  ],
  "summary": "Summary (1-2 sentences)"
}"""

structured_prompt = build_structured_prompt(
    role=ROLE,
    goal=GOAL,
    constraints=CONSTRAINTS,
    format_spec=FORMAT_SPEC,
    context=CONTEXT_CHUNKS,
    question=USER_QUESTION
)

print("=" * 80)
print("구조화 프롬프트:")
print("=" * 80)
print(structured_prompt)
print("=" * 80)


---
## 5️⃣ 프롬프트 비교 실행

이 셀에서는 비구조화 프롬프트와 구조화 프롬프트를 각각 LLM에 전달하여 응답을 비교합니다.

**주요 내용:**
1. 비구조화 프롬프트로 LLM 호출
2. 구조화 프롬프트로 LLM 호출
3. 두 응답을 나란히 비교 출력

**실행 결과:**
- 두 응답이 출력되어 품질과 구조의 차이를 확인할 수 있습니다.
- 구조화 프롬프트는 근거 기반 답변과 인용을 포함하는 반면,
- 비구조화 프롬프트는 일반적인 답변만 제공합니다.

**예상 차이점:**
- 구조화 프롬프트: 컨텍스트 인용, JSON 형식, 근거 명시
- 비구조화 프롬프트: 일반 지식 기반, 자유 형식, 인용 없음


In [ ]:
# ========================================
# 5️⃣ 프롬프트 비교 실행
#    - 비구조화 프롬프트 응답
#    - 구조화 프롬프트 응답
#    - 비교 분석
# ========================================

print("🔄 비구조화 프롬프트로 LLM 호출 중...")
print("-" * 80)
unstructured_response = ollama_generate(unstructured_prompt)
print("\n✅ 비구조화 프롬프트 응답:")
print("=" * 80)
print(unstructured_response)
print("=" * 80)

print("\n\n")

print("🔄 구조화 프롬프트로 LLM 호출 중...")
print("-" * 80)
structured_response = ollama_generate(structured_prompt)
print("\n✅ 구조화 프롬프트 응답:")
print("=" * 80)
print(structured_response)
print("=" * 80)

print("\n\n")

print("=" * 80)
print("📊 비교 분석")
print("=" * 80)
print("\n[비구조화 프롬프트 특징]")
print("- 일반 지식 기반 답변")
print("- 자유 형식의 설명")
print("- 컨텍스트 인용 없음")
print("- 근거 명시 불명확")

print("\n[구조화 프롬프트 특징]")
print("- 컨텍스트 기반 답변")
print("- 구조화된 JSON 형식 (요청 시)")
print("- 명시적 인용 포함")
print("- 근거 명시 필수")

print("\n[핵심 차이점]")
print("- 구조화 프롬프트는 RAG 파이프라인에서 검색된 문서를 효과적으로 활용")
print("- 비구조화 프롬프트는 LLM의 일반 지식에 의존하여 일관성 없는 답변 가능")
print("=" * 80)
